#Tarea: Predicción precios casas - Regresión Lineal con Regularización

# Nombre: Isaac Cueva

# Fecha: 17/05/2025

![Descripción de la imagen](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRP9d63jC144Usqa_NjecnoWcJXV60JuFTJLA&s)

###Inciar una Sesión de Spark

In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [2]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context.
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# Provides findspark.init() to make pyspark importable as a regular library.
import findspark
findspark.init()

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Regresión Lineal") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [4]:
spark

#TAREAS

Con el dataset adjunto, genere un modelo de regresión lineal regularizado que permita predecir el valor de la mediana de una casa en California.

Las variables que están en el dataset son:

1. longitude (signed numeric - float) : Valor de longitud para el bloque en California, EE. UU.
2. latitude (numeric - float ) : Valor de latitud para el bloque en California, EE. UU.
3. housing_median_age (numeric - int ) : Edad media de la casa en el bloque
4. total_rooms (numeric - int ) : Recuento del número total de habitaciones (excluidos los dormitorios) en todas las casas del bloque
5. total_bedrooms (numeric - float ) : Recuento del número total de dormitorios en todas las casas del bloque
6. population (numeric - int ) : Recuento del número total de población en el bloque
7. households (numeric - int ) : Recuento del número total de hogares en el bloque
8. median_income (numeric - float ) : Mediana del ingreso total del hogar de todas las casas del bloque
9. ocean_proximity (numeric - categórico ) : Tipo de paisaje del bloque [ Valores únicos : 'CERCA DE LA BAHÍA', '<1H OCEAN', 'INTERIOR', 'CERCA DEL OCÉANO', 'ISLA' ]
10. median_house_value (numeric - int ) : Mediana de los precios de las viviendas de todas las casas del bloque

Actividades a realizar:

1. Solvente cualquier problema que exista con datos faltantes.

In [5]:

file_path = "1553768847-housing.csv"

# Cargar CSV en Spark DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Mostrar esquema para validar la carga
df.printSchema()

# Mostrar primeras filas para inspección
df.show(5)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: integer (nullable = true)
 |-- total_rooms: integer (nullable = true)
 |-- total_bedrooms: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- households: integer (nullable = true)
 |-- median_income: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)
 |-- median_house_value: integer (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|ocean_proximity|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+
|  -122.23|   37.88|                41|        880|           129|       322|       126|       8.3252|       NEAR BAY|          

In [6]:
from pyspark.sql.functions import col, sum, when

# Contar valores nulos por columna
df.select([sum(when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in df.columns]).show()


+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|ocean_proximity|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|              0|                 0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+---------------+------------------+



- Corregir los nulos con la Mediana

In [9]:
from pyspark.sql.functions import lit

# Calcular la mediana
mediana = df.approxQuantile("total_bedrooms", [0.5], 0.01)[0]

# Imputar valores nulos con la mediana
df = df.na.fill({"total_bedrooms": mediana})

# Verificar que ya no haya nulos en total_bedrooms
df.select(sum(when(col("total_bedrooms").isNull(), 1).otherwise(0)).alias("nulos_total_bedrooms")).show()


+--------------------+
|nulos_total_bedrooms|
+--------------------+
|                   0|
+--------------------+



2. Transforme las variables categóricas en variables tipo dummy.

In [10]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# StringIndexer para la columna categórica
indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")

# OneHotEncoder para transformar índices en variables dummy
encoder = OneHotEncoder(inputCols=["ocean_proximity_index"], outputCols=["ocean_proximity_vec"])

# Crear pipeline para encadenar ambos pasos
pipeline = Pipeline(stages=[indexer, encoder])

# Ajustar y transformar el DataFrame
df_encoded = pipeline.fit(df).transform(df)

# Mostrar resultado con las nuevas columnas
df_encoded.select("ocean_proximity", "ocean_proximity_index", "ocean_proximity_vec").show(5, truncate=False)


+---------------+---------------------+-------------------+
|ocean_proximity|ocean_proximity_index|ocean_proximity_vec|
+---------------+---------------------+-------------------+
|NEAR BAY       |3.0                  |(4,[3],[1.0])      |
|NEAR BAY       |3.0                  |(4,[3],[1.0])      |
|NEAR BAY       |3.0                  |(4,[3],[1.0])      |
|NEAR BAY       |3.0                  |(4,[3],[1.0])      |
|NEAR BAY       |3.0                  |(4,[3],[1.0])      |
+---------------+---------------------+-------------------+
only showing top 5 rows



3. Construir vector de características

In [11]:
from pyspark.ml.feature import VectorAssembler

# Lista columnas numéricas (excluyendo median_house_value)
numeric_cols = ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms",
                "population", "households", "median_income"]

assembler = VectorAssembler(
    inputCols=numeric_cols + ["ocean_proximity_vec"],
    outputCol="features"
)

# Crear nuevo DataFrame con columna features y columna label para la variable objetivo
df_final = assembler.transform(df_encoded).select("features", col("median_house_value").alias("label"))

# Mostrar ejemplo
df_final.show(5, truncate=False)


+-----------------------------------------------------------------------+------+
|features                                                               |label |
+-----------------------------------------------------------------------+------+
|[-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,0.0,0.0,0.0,1.0]    |452600|
|[-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,0.0,0.0,0.0,1.0]|358500|
|[-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,0.0,0.0,0.0,1.0]   |352100|
|[-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,0.0,0.0,0.0,1.0]   |341300|
|[-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,0.0,0.0,0.0,1.0]   |342200|
+-----------------------------------------------------------------------+------+
only showing top 5 rows



4. Divida el dataset en datos de entrenamiento y datos de evaluación.

In [12]:
train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=42)

print(f"Tamaño de entrenamiento: {train_df.count()}")
print(f"Tamaño de evaluación: {test_df.count()}")


Tamaño de entrenamiento: 16560
Tamaño de evaluación: 4080


5. Genere un modelo de regresión multivariable para predecir la mediana de los precios de las casas.
Evalúe su modelo con el MSE y el valor de R^2,

In [13]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Crear el modelo de regresión lineal sin regularización
lr = LinearRegression(featuresCol="features", labelCol="label", regParam=0.0, elasticNetParam=0.0)

# Entrenar el modelo
lr_model = lr.fit(train_df)

# Predecir en el conjunto de test
predictions = lr_model.transform(test_df)

# Evaluadores para MSE y R^2
evaluator_mse = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="mse")
evaluator_r2 = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")

# Calcular métricas
mse = evaluator_mse.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print(f"MSE: {mse}")
print(f"R²: {r2}")


MSE: 5010754518.012139
R²: 0.6378429926191952


6. Genere un modelo de regresión multivariable de grado 2 para predecir la mediana de los precios de las casas.
Evalúe su modelo con el MSE y el valor de R^2.

In [14]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.regression import LinearRegression

# Expansor polinómico grado 2
polyExpansion = PolynomialExpansion(degree=2, inputCol="features", outputCol="polyFeatures")

# Transformar conjuntos de entrenamiento y prueba
train_poly = polyExpansion.transform(train_df)
test_poly = polyExpansion.transform(test_df)

# Modelo de regresión lineal con features polinómicas
lr_poly = LinearRegression(featuresCol="polyFeatures", labelCol="label", regParam=0.0, elasticNetParam=0.0)

# Entrenar modelo
lr_poly_model = lr_poly.fit(train_poly)

# Predecir en test
predictions_poly = lr_poly_model.transform(test_poly)

# Evaluar métricas
mse_poly = evaluator_mse.evaluate(predictions_poly)
r2_poly = evaluator_r2.evaluate(predictions_poly)

print(f"MSE Modelo Polinómico grado 2: {mse_poly}")
print(f"R² Modelo Polinómico grado 2: {r2_poly}")


MSE Modelo Polinómico grado 2: 21057602127.419735
R² Modelo Polinómico grado 2: -0.521958048766564


El resultado indica que el modelo está funcionando peor que una simple predicción basada en la media del valor objetivo. La expansión polinómica genera una cantidad muy grande de variables, lo que provoca sobreajuste y alta complejidad. Además, con muchas variables y sin una adecuada selección o regularización, el modelo se vuelve inestable y no generaliza bien.

7. Seleccione el mejor modelo y aplique regularización: genere un modelo de regesión Lasso.
Evalúe su modelo con el MSE y el valor de R^2.

In [17]:
from pyspark.ml.regression import LinearRegression

# Modelo Lasso (elasticNetParam=1)
lasso = LinearRegression(featuresCol="scaledFeatures", labelCol="label", regParam=0.1, elasticNetParam=1.0)

# Entrenar modelo Lasso con datos escalados de entrenamiento
lasso_model = lasso.fit(train_scaled)

# Predecir en datos de test escalados
predictions_lasso = lasso_model.transform(test_scaled)

# Evaluar con MSE y R2
mse_lasso = evaluator_mse.evaluate(predictions_lasso)
r2_lasso = evaluator_r2.evaluate(predictions_lasso)

print(f"MSE Modelo Lasso: {mse_lasso}")
print(f"R² Modelo Lasso: {r2_lasso}")


MSE Modelo Lasso: 5153100102.186113
R² Modelo Lasso: 0.6275548313067607


8. Aplique nuevamente regularización al mejor modelo: genere un modelo de regresión Ridge.
Evalúe su modelo con el MSE y el valor de R^2.

In [19]:
# Modelo Ridge (elasticNetParam=0)
ridge = LinearRegression(featuresCol="scaledFeatures", labelCol="label", regParam=0.1, elasticNetParam=0.0)

# Entrenar modelo Ridge
ridge_model = ridge.fit(train_scaled)

# Predecir en test
predictions_ridge = ridge_model.transform(test_scaled)

# Evaluar
mse_ridge = evaluator_mse.evaluate(predictions_ridge)
r2_ridge = evaluator_r2.evaluate(predictions_ridge)

print(f"MSE Modelo Ridge: {mse_ridge}")
print(f"R² Modelo Ridge: {r2_ridge}")


MSE Modelo Ridge: 5154029304.250958
R² Modelo Ridge: 0.6274876723513885


9. Concluya cual de los tres modelos (modelo normal y los dos con regularización), es el mejor modelo y porqué.

| Modelo                     | MSE              | R²            |
|----------------------------|------------------|---------------|
| Regresión Lineal Simple     | 5,010,754,518.01 | 0.6378        |
| Regresión Lineal con Lasso  | 5,153,100,102.19 | 0.6276        |
| Regresión Lineal con Ridge  | 5,154,029,304.25 | 0.6275        |


Aunque el modelo lineal simple obtuvo un ajuste ligeramente mejor en términos de MSE y R², los modelos con regularización de Lasso y Ridge presentan una mayor robustez y capacidad de generalización gracias a la penalización aplicada. Entre estos, Lasso es preferible si se busca un modelo más simple y con selección automática de variables, mientras que Ridge es útil para mantener todas las variables con coeficientes regularizados. A mi criterio, concluyo que el mejor es Lasso por su balance entre desempeño y simplicidad, lo que facilita la interpretación y evita el sobreajuste.